In [116]:
import json
import csv
import requests
import os

begin_time = datetime.datetime.now()

prop_lists_json_path = 'prop_lists/json/'
prop_lists_csv_path = 'prop_lists/csv/'
access_token = 'MjY0OTEw|6e27c6250a764bbfa6988a18ea93b4f3'
year = '2019'

city_id_dict = {
    '79752' : 'dallas',
    '79888' : 'fort-worth',
    '80413' : 'plano',
    '80045' : 'irving',
    '79491' : 'arlington',
    '79901' : 'frisco',
    '79949' : 'grand-prairie',
    '80237' : 'mckinney',
    '79771' : 'denton',
    '80482' : 'richardson'
}

markets = {
    'dallas' : 'Dallas',
    'fort-worth' : 'Fort-Worth',
    'plano' : 'Plano',
    'irving' : 'Irving',
    'arlington' : 'Arlington',
    'frisco' : 'Frisco',
    'grand-prairie' : 'Grand Prairie',
    'mckinney' : 'McKinney',
    'denton' : 'Denton',    
    'richardson' : 'Richardson'
}

def setUp():      
    # Setup property listing folders
    if not os.path.exists(prop_lists_json_path):
        os.makedirs(prop_lists_json_path)
    if not os.path.exists(prop_lists_csv_path):
        os.makedirs(prop_lists_csv_path)

def fetch_prop_listing_json_files():    
    for city_id, city_name in city_id_dict.items():
        url='https://api.airdna.co/v1/market/property_list?access_token={}&city_id={}&start_month=1&start_year={}&number_of_months=12&currency=native&show_regions=true'.format(
            access_token, city_id, year
        )
        prop_list_json = requests.get(url)
        data = prop_list_json.content
        file_path = '{}{}_property_list_{}.json'.format(prop_lists_json_path, city_name, year)
        with open(file_path, 'wb') as f:
            f.write(data)

def convert_prop_listing_json_data_to_csv():
    setUp()
    unwanted_prop_keys = ['occ', 'revenue', 'regions', 'id', 'platforms']
    for city_id, city_name in city_id_dict.items():
        file_path = '{}{}_property_list_{}.json'.format(prop_lists_json_path, city_name, year)
        with open(file_path) as f:
            data = json.load(f)
        property_data = data['properties']
        # Clean data and add additional calculated fields
        for prop in property_data:
            # Additional columns
            prop['airbnb_url'] = 'https://www.airbnb.com/rooms/' + str(prop['airbnb_property_id'])
            # Check & record if Airbnb listing is active
            prop['active_listing'] = 0
            if (requests.head(prop['airbnb_url']).status_code == 200):
                prop['active_listing'] = 1
            prop['market'] = markets[city_name]
            # Filter out unwanted property keys
            for unwanted_prop_key in unwanted_prop_keys:
                prop.pop(unwanted_prop_key)

        # Write data to csv
        csv_file_path = '{}{}_property_list_{}.csv'.format(prop_lists_csv_path, city_name, year)
        data_file = open(csv_file_path, 'w', encoding='utf-8', newline='')
        csv_writer = csv.writer(data_file)
        count = 0

        for prop in property_data: 
            if count == 0: 
                # Write headers
                header = prop.keys() 
                csv_writer.writerow(header) 
                count += 1
            # Write data 
            csv_writer.writerow(prop.values()) 

        data_file.close()

# fetch_prop_listing_json_files()
# convert_prop_listing_json_data_to_csv()
# print('Completed')
# print(datetime.datetime.now() - begin_time)